In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
from pylab import rcParams
%pylab inline
rcParams['figure.figsize'] = 12, 6
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.grid_search import GridSearchCV

Populating the interactive namespace from numpy and matplotlib


In [10]:
d = pd.read_csv('tmp/features_02.csv')
d = d[d.TripType_0 == 0]
del d['TripType']
del d['Weekday']
del d['Weekday_nan']
lim = int(0.7 * len(d))
dtr = d[:lim]
dho = d[lim:]
tr = DataFrame({'VisitNumber': dtr.VisitNumber})
trl = DataFrame({'VisitNumber': dtr.VisitNumber})
ho = DataFrame({'VisitNumber': dho.VisitNumber})
hol = DataFrame({'VisitNumber': dho.VisitNumber})
for c in d.columns:
    if (c.startswith('ScanCount') or
        c.startswith('DepartmentDescription') or
        c.startswith('Weekday')):
        tr[c] = dtr[c]
        ho[c] = dho[c]
    if c.startswith('TripType'):
        trl[c] = dtr[c]
        hol[c] = dho[c]
del trl['TripType_0']
del hol['TripType_0']

In [13]:
trl.sum()[:10]

VisitNumber    4486048960
TripType_3           2733
TripType_4            272
TripType_5           3275
TripType_6            880
TripType_7           4002
TripType_8           8557
TripType_9           6644
TripType_12            85
TripType_14             4
dtype: int64

In [19]:
target = 'TripType_6'
param_grid = {
    'n_estimators': [70, 80, 100],
    'max_depth': [4, 5, 6],
    'min_child_weight': [4, 5, 6],
    'learning_rate': [0.02, 0.03, 0.04],
    'objective': ['binary:logistic'],
    'nthread': [4]
}
xgb = XGBClassifier()
m = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    cv=3)
m.fit(tr, trl[target])
print m.best_score_
print m.best_params_

0.992593809261
{'learning_rate': 0.03, 'nthread': 4, 'min_child_weight': 4, 'n_estimators': 80, 'objective': 'binary:logistic', 'max_depth': 5}


In [20]:
def eval_model(m, target):
    m.fit(tr, trl[target])
    p = m.predict_proba(ho)[:,1]
    return roc_auc_score(hol[target], p)

In [41]:
eval_model(XGBClassifier(
    objective='binary:logistic',
    n_estimators=200,
    max_depth=4,
    min_child_weight=4,
    learning_rate=0.03,
    nthread=4
    ),
    target='TripType_3')

0.99563266582659948

In [62]:
pred = DataFrame({'VisitNumber': hol.VisitNumber})
for c in hol.columns:
    if c.startswith('TripType'):
        #print c
        m = XGBClassifier(
            objective='binary:logistic',
            n_estimators=80,
            max_depth=5,
            min_child_weight=4,
            learning_rate=0.03,
            nthread=4)
        m.fit(tr, trl[c])
        p = m.predict_proba(ho)[:,1]
        pred[c] = p

In [64]:
def foo(row):
    x = row.max()
    q = row.quantile(0.9)
    row[row < x] = 0
    return row

In [72]:
pred2 = pred.copy()
vn = pred2.VisitNumber
del pred2['VisitNumber']
pred2 = pred2.apply(foo, axis=1)
y_pred = pred2.copy()
y_pred_norm = y_pred.apply(lambda row: (row / row.sum()), axis=1)
z = y_true * y_pred_norm
z.sum().sum()
# z = z.apply(np.log)
# z = z.replace([-inf], 0)
# print -z.sum().sum()/len(y_true)

19431.0

In [71]:
pred2 = pred.copy()
vn = pred2.VisitNumber
del pred2['VisitNumber']
pred2 = pred2.apply(foo, axis=1)
y_true = hol.copy()
del y_true['VisitNumber']
y_pred = pred2.copy()
y_pred_norm = y_pred.div(pred.sum(axis=1), axis=0)
z = y_true * y_pred_norm
z = z.apply(np.log)
z = z.replace([-inf], 0)
print -z.sum().sum()/len(y_true)

8.47486757732
